# Ejemplo del laberinto. Implementación del algoritmo de Evaluación de Políticas.

Vamos a revisar esta implementación del ejercicio del laberinto que hemos visto en las diapositivas y que procede del libro de Sutton & Barto. Aquí vamos a utilizar la convergencia iterativa de las ecuaciones de Bellman para resolver el mismo ejercicio del laberinto.

In [1]:
#######################################################################
# Copyright (C)                                                       #
# 2016-2018 Shangtong Zhang(zhangshangtong.cpp@gmail.com)             #
# 2016 Kenta Shimada(hyperkentakun@gmail.com)                         #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################


 Definimos las estructuras de datos:  
 * Definimos las coordenadas de las posiciones especiales A, B, A', B'. 
   Ten en cuenta que los índices comienzan por 0 en python
 * Definimos  la variable DISCOUNT que es nuestra Gamma de teoria a un valor 0.9
 * La variable WORL_SIZE nos define un tablero o matriz de 5x5
 * El vector Actions define con dos códigos el cambio de índices en el tablero al moverse 
   ejemplo: estoy en [2,2], elijo la accion 'left', con  lo que sumo [2,2] + [0,-1] = [2,1] y estoy 
   una posición a la izquierda
 * La constante ACTION_PROB es la probabilidad de cada acción
 
    -

In [1]:

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.table import Table

matplotlib.use('Agg')

WORLD_SIZE = 5
A_POS = [0, 1]
A_PRIME_POS = [4, 1]
B_POS = [0, 3]
B_PRIME_POS = [2, 3]
DISCOUNT = 0.9

# left, up, right, down
ACTIONS = [np.array([0, -1]),
           np.array([-1, 0]),
           np.array([0, 1]),
           np.array([1, 0])]
ACTIONS_FIGS=[ '←', '↑', '→', '↓']


ACTION_PROB = 0.25

La siguiente función 'step' implementa el paso elemental $s_t,a_t,r_{t+1},s_{t+1},a_{t+1}$
Si el estado actual es 'A' o 'B', salto a los puntos 'A'' o 'B'' y devuelvo la recompensa +10 ó +5 
Sino, hago el movimiento, sumándole al estado la acción (como hemos explicado antes) y compruebo si
me he salido fuera del tablero.
    -Si he salido, devuelvo -1 y vuelvo al estado anterior
    -Si no he salido, devuelvo 0

In [2]:
def step(state, action):
    if state == A_POS:
        return A_PRIME_POS, 10
    if state == B_POS:
        return B_PRIME_POS, 5

    next_state = (np.array(state) + action).tolist()
    x, y = next_state
    if x < 0 or x >= WORLD_SIZE or y < 0 or y >= WORLD_SIZE:
        reward = -1.0
        next_state = state
    else:
        reward = 0
    return next_state, reward


Las dos funciones (celdas de código) siguientes no tienen interés algorítmico. Son para dibujar la imagen de los resultados del tablero.

In [3]:
def draw_image(image):
    fig, ax = plt.subplots()
    ax.set_axis_off()
    tb = Table(ax, bbox=[0, 0, 1, 1])

    nrows, ncols = image.shape
    width, height = 1.0 / ncols, 1.0 / nrows

    # Add cells
    for (i, j), val in np.ndenumerate(image):

        # add state labels
        if [i, j] == A_POS:
            val = str(val) + " (A)"
        if [i, j] == A_PRIME_POS:
            val = str(val) + " (A')"
        if [i, j] == B_POS:
            val = str(val) + " (B)"
        if [i, j] == B_PRIME_POS:
            val = str(val) + " (B')"
        
        tb.add_cell(i, j, width, height, text=val,
                    loc='center', facecolor='white')
        

    # Row and column labels...
    for i in range(len(image)):
        tb.add_cell(i, -1, width, height, text=i+1, loc='right',
                    edgecolor='none', facecolor='none')
        tb.add_cell(-1, i, width, height/2, text=i+1, loc='center',
                    edgecolor='none', facecolor='none')

    ax.add_table(tb)

In [4]:
def draw_policy(optimal_values):
    fig, ax = plt.subplots()
    ax.set_axis_off()
    tb = Table(ax, bbox=[0, 0, 1, 1])

    nrows, ncols = optimal_values.shape
    width, height = 1.0 / ncols, 1.0 / nrows

    # Add cells
    for (i, j), val in np.ndenumerate(optimal_values):
        next_vals=[]
        for action in ACTIONS:
            next_state, _ = step([i, j], action)
            next_vals.append(optimal_values[next_state[0],next_state[1]])

        best_actions=np.where(next_vals == np.max(next_vals))[0]
        val=''
        for ba in best_actions:
            val+=ACTIONS_FIGS[ba]
        
        # add state labels
        if [i, j] == A_POS:
            val = str(val) + " (A)"
        if [i, j] == A_PRIME_POS:
            val = str(val) + " (A')"
        if [i, j] == B_POS:
            val = str(val) + " (B)"
        if [i, j] == B_PRIME_POS:
            val = str(val) + " (B')"
        
        tb.add_cell(i, j, width, height, text=val,
                loc='center', facecolor='white')

    # Row and column labels...
    for i in range(len(optimal_values)):
        tb.add_cell(i, -1, width, height, text=i+1, loc='right',
                    edgecolor='none', facecolor='none')
        tb.add_cell(-1, i, width, height/2, text=i+1, loc='center',
                   edgecolor='none', facecolor='none')

    ax.add_table(tb)

### Este es el meollo
La función 'figure_3_2()' implementa el algoritmo 'Iterative Policy Evaluation'
La función figure_3_5() implementa el método de optimización del algoritmo 'Value Iteration'

In [5]:
#Esta función EVALUA la política consistente en asignar 0.25 de probabilidad cada acción
def figure_3_2():
    value = np.zeros((WORLD_SIZE, WORLD_SIZE))
    while True:
        # keep iteration until convergence
        new_value = np.zeros_like(value)
        for i in range(WORLD_SIZE):
            for j in range(WORLD_SIZE):
                for action in ACTIONS:
                    (next_i, next_j), reward = step([i, j], action)
                    # Backup de la función de Bellman para la politica descrita
                    new_value[i, j] += ACTION_PROB * (reward + DISCOUNT * value[next_i, next_j])
        if np.sum(np.abs(value - new_value)) < 1e-4:
            draw_image(np.round(new_value, decimals=2))
            plt.savefig('imagesfigure_3_2.png')
            plt.close()
            break
        value = new_value


#Value Iteration. Esta función es UNA OPTIMIZACION, es decir , encuentra la política óptima
def figure_3_5():
    value = np.zeros((WORLD_SIZE, WORLD_SIZE))
    while True:
        # keep iteration until convergence
        new_value = np.zeros_like(value)
        for i in range(WORLD_SIZE):
            for j in range(WORLD_SIZE):
                values = []
                for action in ACTIONS:
                    (next_i, next_j), reward = step([i, j], action)
                    # value iteration
                    #Fijaros que aquí estamos guardandonos todos los backups 
                    #correspondientes a las 4 acciones. NO ACTUALIZAMOS LA FUNCION DE VALOR TODAVIA
                    values.append(reward + DISCOUNT * value[next_i, next_j])
                    
                #ESTA INSTRUCCION RECOGE EL MAXIMO DE LOS POSIBLES RETORNOS. 
                #ESTA INSTRUCCION ES LA QUE HACE POSIBLE LA OPTIMIZACION
                new_value[i, j] = np.max(values)
        if np.sum(np.abs(new_value - value)) < 1e-4:
            draw_image(np.round(new_value, decimals=2))
            plt.savefig('figure_3_5.png')
            plt.close()
            draw_policy(new_value)
            plt.savefig('figure_3_5_policy.png')
            plt.close()
            break
        value = new_value


In [6]:
if __name__ == '__main__':
   
    figure_3_2()
    #figure_3_5()



### Ejercicio 1.
Completa el código siguiente para implementar el algoritmo de Policy Iteration que maximiza la funcion de valor. Este algoritmo esta separado en dos partes: 
- 1. Policy evaluation (que obtiene la funcion de valor de la politica actual en cada iteracion) Nota: Daros cuenta que la politica actual CAMBIA en cada iteracion debido a la segunda parte del algoritmo.
- 2. Policy Improvement (que construye una politica greedy a partir de la funcion de valor actual)

In [17]:
def policy_iteration():
    value = np.zeros((WORLD_SIZE, WORLD_SIZE))
    #Ahora, como la politica no es constante como en "figure_3_2()", debemos implementar una
    #estructura de datos para representar a la politica en cada iteracion.
    #Esta estructura es una matriz de tamaño WORLD_SIZEx WORLD_SIZE de enteros donde
    #almacenaremos los indices de la accion para cada estado {0,1,2,3}
    #Igual que la instruccción de arriba carga en 'policy una matriz de enteros con 
    #valores aleatorios de 0 a 3 en cada casilla
    policy = ??
    
    #bandera booleana que nos permite acabar con el bucle que le sigue
    policy_estable = False
    while policy_estable == False:
        # keep iteration until convergence
        #---Comentario del Ejercicio 2---
        #Parte 1 de Policy_Evaluation para la evaluación de la politica definida en 'policy'
        new_value = np.zeros_like(value)
        for i in range(WORLD_SIZE):
            for j in range(WORLD_SIZE):
                
                #ATENCION: Aqui ya hay una diferencia con la policy evaluation de "figure_3_2()"
                #La accion la escogemos de la matriz 'policy', porque cambia en cada iteracion
                #Usa el indice almacenado en 'policy' para elegir la accion de 'ACTIONS'
                action = ??  #pi(s)
                
                #Ejecutamos ahora el paso usando la función 'step' con la acción elegida
                (next_i, next_j), reward = ??
                # Actuaiza el backup de la ecuación de Bellman
                #Fijate que usamos otra estructura para almacenar el valor nuevo
                #porque necesitamos conservar la vieja función de valor hasta que todos
                #los valores se hayan actualizado correctamente
                
                #No uses ACTION_PROB en esta actualizacion porque ESTAMOS ELIGIENDO LA POLITICA GREEDY
                new_value[i, j] = ??
        if np.sum(np.abs(value - new_value)) < 1e-4:
            #draw_image(np.round(new_value, decimals=2))
            #plt.savefig('imagesfigure_3_2.png')
            #plt.close()
            break
        #Aqui actualizo la función de valor    
        value = new_value
      

    #Parte 2 de policy_Improvement para la mejora de la politica
    
    
        # keep iteration until convergence
        policy_estable = True
        
        for i in range(WORLD_SIZE):
            for j in range(WORLD_SIZE):
                b = policy[i,j]
                #Ahora me quedo con la accion que maximiza la ecuacion de Bellman
                #Nota: El codigo que sigue esta pensado en C++. No sigue la 
                #filosofia de python
                #¿Podrias tú hacerlo mejor?
                #Basta con almacenar en un vector los cálculos de los valores de (s,a_i)
                #y luego asignar a la politica el índice de la acción del valor máximo
                
                
                max_action_value =-1000 #inicializo el valor de accion a uno muy bajo
                for action in ACTIONS:
                    #Ejecuta el paso desde 's_ij' con esta acción
                    (next_i, next_j), reward = ??
                    
                    #Calcula el retorno de esa acción (la formula la misma que en la parte 1)
                    aux = ??
                    
                    #Esto es lo que debes de cambiar, si te parece cutre. Sino, el codigo funciona
                    if aux > max_action_value:
                        max_action_value = aux
                        #Guardo en la politica el indice de la accion  máxima {o0, o1, o2, o3}
                        policy[i,j] = next((i for i, val in enumerate(ACTIONS) if np.all(val == action)), -1)  
                    
                #Condición de parada    
                if b != policy[i,j]:
                    policy_estable = False
                    
       
     #volvemos al while inicial               
                    
    draw_image(np.round(value, decimals=2))
    plt.savefig('images_policy_iteration.png')
    plt.close()
    draw_policy(value)
    plt.savefig('figure_policy_iteration.png')
    plt.close()

In [18]:
#Aqui deberias comentar la función main que esta dos cajas más arriba y ponerla aquí para ejecutar tu nueva función

#Igual que antes, esta función te genera dos imagenes, una con la funcion de valor máxima y otra con la 
#política óptima. Compara la solución con la proporcionada con el algoritmo de value iteration
    policy_iteration()

### Ejercicio 2.
Si te fijas en el resultado que te da el algoritmo implementado "policy_iteration()"  de la caja anterior, observarás que,aunque se aproxima al verdadero valor óptimo de cada estado, no llega a alcanzar totalmente el verdadero valor óptimo (compáralo con la diapo 67 de la presentación donde está la solución). Esto es debido a que la parte del algoritmo donde se calcula el valor de V para la política actual no se calcula más que mediante una única iteración sobre los estados, por eficiencia. Esto es posible hacerlo ya que el algoritmo garantiza igualmente la convergencia.  Modifica esta parte (señalada como ---Comentario del Ejercicio 2---) para que sea igual que la función "figure_3_2()" que implementa el cálculo más aproximado y compara los resultados que obtienes ahora con el anterior.